In [ ]:
import pandas as pd

df = pd.read_csv('Corona_NLP_train.csv', encoding = "ISO-8859-1")



In [ ]:
df.count()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
...,...,...,...,...,...,...
41152,44951,89903,"Wellington City, New Zealand",14-04-2020,Airline pilots offering to stock supermarket s...,Neutral
41153,44952,89904,NaN,14-04-2020,Response to complaint not provided citing COVI...,Extremely Negative
41154,44953,89905,NaN,14-04-2020,You know itÂs getting tough when @KameronWild...,Positive
41155,44954,89906,NaN,14-04-2020,Is it wrong that the smell of hand sanitizer i...,Neutral


In [51]:
df_new = df[['OriginalTweet', 'Sentiment']]

In [52]:
# baixando as libs para pre processamento

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

nltk.download('stopwords')

stopwords = set(stopwords.words('english'))

#stemmer = PorterStemmer()
nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/developer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/developer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [53]:
# pre processamento

    # Remover caracteres especiais e números
    #text = re.sub('[^A-Za-z\s\']+', '', text)

def preprocess_text(text):

    padrao_arroba = re.compile(r'@\S+')
      # Remover as strings com "@" no início da string
    text = re.sub(padrao_arroba, '', text)

        #remover sites
    #pattern_url = re.compile(r'https?:\/\/.*?[\s+]')

    text = re.sub(r'http\S+', '', text)
    

    # Converter tudo para letras minúsculas
    text = text.lower()

    
    # Remover as tags HTML
    text = re.sub('<[^<]+?>', '', text)    
    
    
    # Tokenizar o texto em palavras
    words = nltk.word_tokenize(text)

    # remover palavras que so possua caractere especial
    words = [word for word in words if not re.match('^[^A-Za-z0-9]+$', word)]

    # remover palavras que tenha tamanho 1
    words = [word for word in words if len(word) > 2]

    
    # Remover as stopwords
    words = [word for word in words if word not in stopwords]
    
    # Aplicar o stemming
    #words = [stemmer.stem(word) for word in words]
    words = [lemmatizer.lemmatize(word) for word in words]

    # Juntar as palavras em uma única string
    text = ' '.join(words)
  
    return text

In [54]:
# rodar o pre processamento

def preprocess_collection(docs):

    preprocessed_documents = []

    #for document in docs:
    for index, row in docs.iterrows():
        tema = row['Sentiment']
        #nome_arquivo = document['nome_arquivo']
        conteudo = row['OriginalTweet']
        
        preprocessed_conteudo = preprocess_text(conteudo)
        
        preprocessed_document = {
            'tema': tema,
        #    'nome_arquivo': nome_arquivo,
            'conteudo': preprocessed_conteudo
        }
        
        preprocessed_documents.append(preprocessed_document)
        
    return preprocessed_documents

In [55]:
df_new.head(4)

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,advice Talk to your neighbours family to excha...,Positive
2,Coronavirus Australia: Woolworths to give elde...,Positive
3,My food stock is not the only one which is emp...,Positive


In [57]:
df_new = df_new.sort_values('Sentiment')

In [58]:
preprocessed_documents = preprocess_collection(df_new)


In [59]:
preprocessed_documents

[{'tema': 'Extremely Negative',
  'conteudo': 'panic shopping covid-19 planning end food water toilet paper consumer also panic shopping life insurance take minute learn'},
 {'tema': 'Extremely Negative',
  'conteudo': 'ceo interviewed alongside several agency leader discus evolving medium landscape brand adapting current crisis check'},
 {'tema': 'Extremely Negative',
  'conteudo': 'use scarf wear snood supermarket offer zero protection virus aerosol wear touch face distance hygiene better scarf listen idiot'},
 {'tema': 'Extremely Negative',
  'conteudo': 'sadly everyone good citizen scammer covid-19 staysafe'},
 {'tema': 'Extremely Negative',
  'conteudo': 'god virus would target twat hoarding panic buying unfortunately seems twat winning tho karma bitch coronavirus stoppanicbuying'},
 {'tema': 'Extremely Negative',
  'conteudo': 'north carolina state emergency price gouging officially crime see'},
 {'tema': 'Extremely Negative',
  'conteudo': 'breaking germany biggest department st

In [60]:
frases = [o['conteudo'] for o in preprocessed_documents]

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [61]:
# vetorizando

#temas = [doc["tema"] for doc in preprocessed_documents]
temas = []
for doc in preprocessed_documents:
    temas.append(doc['tema'])


conteudos = [doc["conteudo"] for doc in preprocessed_documents]

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(conteudos)

In [62]:
len(temas)

41157

In [63]:
len(conteudos)

41157

In [3]:
#dataset de teste

import pandas as pd

df_test = pd.read_csv('data/Corona_NLP_test.csv', encoding = "ISO-8859-1")
df_test_new = df_test[['OriginalTweet', 'Sentiment']]

In [4]:
df_test_new

,OriginalTweet,Sentiment
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,When I couldn't find hand sanitizer at Fred Me...,Positive
2,Find out how you can protect yourself and love...,Extremely Positive
3,#Panic buying hits #NewYork City as anxious sh...,Negative
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral
...,...,...
3793,Meanwhile In A Supermarket in Israel -- People...,Positive
3794,Did you panic buy a lot of non-perishable item...,Negative
3795,Asst Prof of Economics @cconces was on @NBCPhi...,Neutral
3796,Gov need to do somethings instead of biar je r...,Extremely Negative


In [65]:
df_test_sorted = df_test_new.sort_values('Sentiment')

In [66]:
doc_consultas_preproc = preprocess_collection(df_test_sorted)

In [32]:
doc_consultas_preproc[600]

{'tema': 'Extremely Positive',
 'conteudo': 'grocery store worker also front lines- thankful need protect coronavirus protecting employee serving absolutely necessary role society'}

In [67]:
array_temas = ['Extremely Negative',
'Extremely Positive',
'Negative',
'Neutral',
'Positive']
result_final = {}

for d in array_temas:
    result_final[d] = {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}

print(result_final)    

{'Extremely Negative': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'Extremely Positive': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'Negative': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'Neutral': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'Positive': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}}


In [68]:
def calcula_map(resultados, tema_verificado):
    acertos = 0
    maps = []
    #for i, doc in enumerate(ranking):
    for i, (similaridade, tema, conteudo) in enumerate(resultados, start=1):
        if tema_verificado == tema:
            acertos +=1
            maps.append(acertos/(i+1))
    return mean(maps)

In [70]:
from statistics import mean

#input_documento = doc_consultas_preproc[150]['conteudo']

p10 = []
p20 = []
p50 = []
p100 = []
map_f = []
tema_atual = ""
for c, input_documento in enumerate(doc_consultas_preproc):
    #print(c)
    if c == 0:
        tema_atual = input_documento['tema']
    #print(tema_atual)
    acertos = 0     

    input_tfidf = vectorizer.transform([input_documento['conteudo']])
    similaridades = cosine_similarity(input_tfidf, tfidf_matrix)[0]

    resultados = list(zip(similaridades, temas, conteudos))
    resultados.sort(reverse=True)
    #print(len(resultados))
    map_f.append(calcula_map(resultados, input_documento['tema']))
    for rank, (similaridade, tema, conteudo) in enumerate(resultados, start=1):
        #print(rank)
        if input_documento['tema'] == tema:
            #print("acertou")
            acertos += 1
            
        if rank == 10:
            #print("deu 10")
            p10.append(acertos/10.0)
            #print(p10)
        if rank == 20:
            p20.append(acertos/20.0)
        if rank == 50:
            p50.append(acertos/50.0)
        if rank == 100:
            p100.append(acertos/100.0)
        
    
    if tema_atual != input_documento['tema'] or c == len(doc_consultas_preproc) - 1:
        print("trocou")
#        print(p10)
        result_final[tema_atual]['p10'] = mean(p10)
        result_final[tema_atual]['p20'] = mean(p20)
        result_final[tema_atual]['p50'] = mean(p50)
        result_final[tema_atual]['p100'] = mean(p100)     
        result_final[tema_atual]['map'] = mean(map_f)
        p10 = []
        p20 = []
        p50 = []
        p100 = []
        map_f = []           

    tema_atual = input_documento['tema']        


trocou
trocou
trocou
trocou
trocou


In [71]:
result_final

{'Extremely Negative': {'p10': 0.20252951096121416,
  'p20': 0.21070826306913998,
  'p50': 0.21780775716694772,
  'p100': 0.216070826306914,
  'map': 0.14139379046747855},
 'Extremely Positive': {'p10': 0.22904841402337228,
  'p20': 0.23013355592654425,
  'p50': 0.228881469115192,
  'p100': 0.2276627712854758,
  'map': 0.16659945794265116},
 'Negative': {'p10': 0.26906820365033624,
  'p20': 0.2656099903938521,
  'p50': 0.2586359269932757,
  'p100': 0.2568876080691643,
  'map': 0.23652568349428577},
 'Neutral': {'p10': 0.4253634894991922,
  'p20': 0.3901453957996769,
  'p50': 0.351437802907916,
  'p100': 0.32221324717285943,
  'map': 0.2080266999543273},
 'Positive': {'p10': 0.28520084566596193,
  'p20': 0.28562367864693444,
  'p50': 0.28788583509513743,
  'p100': 0.2875158562367865,
  'map': 0.3413599605481707}}

In [72]:
import json
with open('results/covid_tfidf.json', 'w') as f:
    json.dump(result_final, f)

In [73]:
import pandas as pd

df = pd.DataFrame(result_final)

In [74]:
df =df.transpose()

In [76]:
df

,p10,p20,p50,p100,map
Extremely Negative,0.202530,0.210708,0.217808,0.216071,0.141394
Extremely Positive,0.229048,0.230134,0.228881,0.227663,0.166599
Negative,0.269068,0.265610,0.258636,0.256888,0.236526
Neutral,0.425363,0.390145,0.351438,0.322213,0.208027
Positive,0.285201,0.285624,0.287886,0.287516,0.341360


In [75]:
df.describe()

,p10,p20,p50,p100,map
count,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.282242,0.276444,0.268930,0.262070,0.218781
std,0.086385,0.069992,0.053596,0.043564,0.077730
min,0.202530,0.210708,0.217808,0.216071,0.141394
25%,0.229048,0.230134,0.228881,0.227663,0.166599
50%,0.269068,0.265610,0.258636,0.256888,0.208027
75%,0.285201,0.285624,0.287886,0.287516,0.236526
max,0.425363,0.390145,0.351438,0.322213,0.341360
